In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torchmetrics

from models import ANN
from model_selection import KFoldCV

In [2]:
X_df = pd.read_csv('trn_X.csv', index_col='Id')
y_df = pd.read_csv('trn_y.csv', index_col='Id')

X, y = torch.tensor(X_df.to_numpy(dtype=np.float32)), torch.tensor(y_df.to_numpy(dtype=np.float32))

In [3]:
X_df

,0,1,2,3,4,5,6,7,8,9,...,287,288,289,290,291,292,293,294,295,296
Id,,,,,,,,,,,,,,,,,,,,,
1,0.235294,0.033420,0.666667,0.500,0.949275,0.883333,0.125089,0.000000,0.064212,0.140098,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.000000,0.038795,0.555556,0.875,0.753623,0.433333,0.173281,0.000000,0.121575,0.206547,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.235294,0.046507,0.666667,0.500,0.934783,0.866667,0.086109,0.000000,0.185788,0.150573,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.294118,0.038561,0.666667,0.500,0.311594,0.333333,0.038271,0.000000,0.231164,0.123732,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
5,0.235294,0.060576,0.777778,0.500,0.927536,0.833333,0.116052,0.000000,0.209760,0.187398,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,0.235294,0.030929,0.555556,0.500,0.920290,0.833333,0.000000,0.000000,0.407962,0.155974,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1457,0.000000,0.055505,0.555556,0.625,0.768116,0.633333,0.139972,0.110583,0.252140,0.252373,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1458,0.294118,0.036187,0.666667,1.000,0.500000,0.933333,0.048724,0.000000,0.375428,0.188543,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [4]:
model_kwargs = {
  'input_dim': X.shape[-1],
  'hidden_dim': [128, 512, 128],
  'drop_ratio': 0.3,
}

dl_kwargs = {
  'batch_size': 36,
  'shuffle': True,
}

optim_kwargs = {
  'lr': 1.0,
}

device = 'cpu'

cv = KFoldCV(X, y, ANN, model_kwargs=model_kwargs,
            epochs=300,
            criterion=F.mse_loss,
            Optimizer=torch.optim.Adam,
            optim_kwargs=optim_kwargs,
            trn_dl_kwargs=dl_kwargs, val_dl_kwargs=dl_kwargs,
            metric=torchmetrics.MeanSquaredError(squared=False).to(device),
            device=device)


In [5]:
res = cv()
res = pd.concat([res, res.apply(['mean', 'std'])])

res

ANN(
  (Identity): Identity()
  (dropout): Dropout(p=0.3, inplace=False)
  (activation): Sigmoid()
  (module_list): ModuleList(
    (0): Linear(in_features=297, out_features=128, bias=True)
    (1): Identity()
    (2): Sigmoid()
    (3): Linear(in_features=128, out_features=512, bias=True)
    (4): Identity()
    (5): Sigmoid()
    (6): Linear(in_features=512, out_features=128, bias=True)
    (7): Identity()
    (8): Sigmoid()
    (9): Linear(in_features=128, out_features=1, bias=True)
    (10): Identity()
  )
)


  0%|          | 0/300 [00:00<?, ?it/s]

/home/sdk/miniconda3/envs/MathAI/lib/python3.10/site-packages/torch/autograd/__init__.py:251: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


KeyboardInterrupt: 

True